# Rework project description column

Goal is to add to the description of projects with multiple funding programs. Current format just picks one.

In [1]:
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase


import _data_utils
import _script_utils

import numpy as np

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

In [3]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [4]:
GCS_FILE_NAME = 'FMIS_Projects_Universe_IIJA_Reporting_062923_ToDLA.xlsx'

### Testing script updated to this file from 6-29-2023

In [5]:
locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))


In [6]:
locodes

,agency_locode,agency_name,district,county_name,rtpa_name,mpo_name,mpo_locode_fads,active_e76s______7_12_2021_
0,6302,Humboldt Bay Harbor Recreation & Conservation District,1,Humboldt County,Humboldt County Association of Governments,NON-MPO,NON-MPO,NaN
1,6330,Willow Creek Community Services District,1,Humboldt County,Humboldt County Association of Governments,NON-MPO,NON-MPO,NaN
2,5036,Trinidad,1,Humboldt County,Humboldt County Association of Governments,NON-MPO,NON-MPO,NaN
3,5049,Ukiah,1,Mendocino County,Mendocino Council of Governments,NON-MPO,NON-MPO,Yes
4,5082,Willits,1,Mendocino County,Mendocino Council of Governments,NON-MPO,NON-MPO,NaN
...,...,...,...,...,...,...,...,...
1036,5465,Leave Blank,53,NaN,NON-RTPA,NON-MPO,NON-MPO,NaN
1037,6250,U.S. Fish and Wildlife Service,53,NaN,NON-RTPA,NON-MPO,NON-MPO,NaN
1038,6031,Tidewater Southern Railway Company,53,NaN,NON-RTPA,NON-MPO,NON-MPO,NaN
1039,6176,Tri-Counties Regional Park Group,53,NaN,NON-RTPA,NON-MPO,NON-MPO,NaN


In [7]:
new_locodes = to_snakecase(pd.read_csv(f"{GCS_FILE_PATH}/program_codes/LP2000_agencies_view.csv")) 

In [8]:
new_locodes

,agency_code,agency_name,agencyid,last_update
0,5001,San Francisco,City of San Francisco,11/9/2023 4:59am
1,5002,Sacramento,City of Sacramento,11/9/2023 4:59am
2,5003,Benicia,City of Benicia,11/9/2023 4:59am
3,5004,San Diego,City of San Diego,11/9/2023 4:59am
4,5005,San Jose,City of San Jose,11/9/2023 4:59am
...,...,...,...,...
915,7502,Hoopa Valley Tribe,"""Hoopa Valley Tribe, California""",11/9/2023 4:59am
916,7503,San Diego Metropolitan Transit System,San Diego Metropolitan Transit System,11/9/2023 4:59am
917,7504,San Pasqual Band of Mission Indians,San Pasqual Band of Diegueno Mission Indians of California,11/9/2023 4:59am
918,7505,Gold Coast Transit District,Gold Coast Transit District,11/9/2023 4:59am


In [9]:
all_locodes = pd.merge(locodes, new_locodes, left_on='agency_locode', right_on= "agency_code", how = 'outer', indicator=True)

In [10]:
all_locodes>>filter(_._merge!="both")>>select(_.agency_locode, _.agency_name_x, _.agency_code, _.agency_name_y, _.agencyid)

,agency_locode,agency_name_x,agency_code,agency_name_y,agencyid
21,6201.0,Caltrans,NaN,NaN,NaN
26,6063.0,Eureka Southern Railroad Company-DO NOT USE see 6242,NaN,NaN,NaN
29,6400.0,Lake County Air Basin,NaN,NaN,NaN
33,6487.0,Humboldt County Resource Conservation,NaN,NaN,NaN
34,6500.0,Mendocino County Health and Human Services Agency,NaN,NaN,NaN
...,...,...,...,...,...
1039,6176.0,Tri-Counties Regional Park Group,NaN,NaN,NaN
1040,6185.0,Trust for Public Land,NaN,NaN,NaN
1041,NaN,NaN,7504.0,San Pasqual Band of Mission Indians,San Pasqual Band of Diegueno Mission Indians of California
1042,NaN,NaN,7505.0,Gold Coast Transit District,Gold Coast Transit District


In [26]:
no_match_x = all_locodes>>filter(_._merge!="both")>>select(_.agency_locode, _.agency_name_x)
no_match_x = no_match_x[no_match_x['agency_name_x'].notnull()]

In [29]:
no_match_x

,agency_locode,agency_name_x
21,6201.0,Caltrans
26,6063.0,Eureka Southern Railroad Company-DO NOT USE see 6242
29,6400.0,Lake County Air Basin
33,6487.0,Humboldt County Resource Conservation
34,6500.0,Mendocino County Health and Human Services Agency
...,...,...
1036,5465.0,Leave Blank
1037,6250.0,U.S. Fish and Wildlife Service
1038,6031.0,Tidewater Southern Railway Company
1039,6176.0,Tri-Counties Regional Park Group


In [24]:
no_match_y = all_locodes>>filter(_._merge!="both")>>select(_.agency_code, _.agency_name_y)
no_match_y = no_match_y[no_match_y['agency_name_y'].notnull()]

In [25]:
no_match_y

,agency_code,agency_name_y
1041,7504.0,San Pasqual Band of Mission Indians
1042,7505.0,Gold Coast Transit District
1043,7506.0,Stanislaus Regional Transit Authority


In [30]:
second_match = pd.merge(no_match_x, no_match_y, left_on='agency_name_x', right_on= "agency_name_y", how = 'outer', indicator=True)

In [31]:
second_match>>filter(_._merge=="both")

,agency_locode,agency_name_x,agency_code,agency_name_y,_merge


In [32]:
second_match>>filter(_._merge!="both")

,agency_locode,agency_name_x,agency_code,agency_name_y,_merge
0,6201.0,Caltrans,NaN,NaN,left_only
1,6202.0,Caltrans,NaN,NaN,left_only
2,6203.0,Caltrans,NaN,NaN,left_only
3,6204.0,Caltrans,NaN,NaN,left_only
4,6205.0,Caltrans,NaN,NaN,left_only
...,...,...,...,...,...
122,6176.0,Tri-Counties Regional Park Group,NaN,NaN,left_only
123,6185.0,Trust for Public Land,NaN,NaN,left_only
124,NaN,NaN,7504.0,San Pasqual Band of Mission Indians,right_only
125,NaN,NaN,7505.0,Gold Coast Transit District,right_only


In [13]:
# df_all = (pd.merge(df, locodes, left_on='locode', right_on='agency_locode', how='left'))


In [5]:
df = _script_utils._prep_data(GCS_FILE_NAME)

In [6]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
619,2022-07-06,Y001,National Highway Performance Program (NHPP),P118068,0716000083S,"IN THE CITY OF LOS ANGELES, FROM TOPANGA CANYON BOULEVARD TO SEPULVEDA BOULEVARD. CONSTRUCT AND INSTALL STORMWATER TREATMENT BEST MANAGEMENT PRACTIC",37,Cong Dist 25,Active,"ON STATE ROUTE: 118. IN THE CITY OF LOS ANGELES, FROM TOPANGA CANYON BOULEVARD TO SEPULVEDA BOULEVARD. CONSTRUCT AND INSTALL STORMWATER TREATMENT BEST MANAGEMENT PRACTICES (BMPS) INCLUDING BIOFILTRATION SWALES/STRIPS, DETENTION BASIN, MEDIA FILTERS AND A GROSS SOLIDS REMOVAL DEVICES (GSRDS) CONSTRUCT AND INSTALL STORMWATER TREATMENT BEST MANAGEMENT PRACTICES (BMPS) INCLUDING BIOFILTRATION SWALES/STRIPS, DETENTION BASIN, MEDIA FILTERS AND A GROSS SOLIDS REMOVAL DEVICES (GSRDS)",6,4R - Restoration & Rehabilitation,634060.82,539571.73,S SCAG,NaN,California,7.0,Los Angeles County,NaN,NaN


### New Program Codes

Now in script

In [7]:
## need to get the program descriptions for these codes
df>>filter(_.program_code_description.isna())>>count(_.program_code)

,program_code,n


In [8]:
## comparing codes: 

code1 = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/program_codes/FY21-22ProgramCodesAsOf5-25-2022.v2_expanded090823.xlsx"))
code1 = code1>>select(_.iija_program_code, _.new_description)
code2 = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/program_codes/Copy of lst_IIJA_Code_20230908.xlsx"))
code2 = code2>>select(_.iija_program_code, _.description, _.program_name)

In [9]:
code_merge = pd.merge(code1, code2, on='iija_program_code', how = 'outer', indicator=True)

In [10]:
code_merge['new_description'] = code_merge['new_description'].str.strip()

In [11]:
compare_col = np.where(code_merge["new_description"] == code_merge["program_name"], True, False)

In [12]:
code_merge["compare_descriptions_to_programname"] = compare_col
code_merge.compare_descriptions_to_programname.value_counts()

True     79
False    57
Name: compare_descriptions_to_programname, dtype: int64

In [13]:
compare_col2 = np.where(code_merge["new_description"] == code_merge["description"], True, False)
code_merge["compare_descriptions_cols"] = compare_col2
code_merge.compare_descriptions_cols.value_counts()

True     87
False    49
Name: compare_descriptions_cols, dtype: int64

In [14]:
code_merge>>filter(_.compare_descriptions_cols==False, _._merge=="both", _.compare_descriptions_to_programname==False)

,iija_program_code,new_description,description,program_name,_merge,compare_descriptions_to_programname,compare_descriptions_cols
1,Y002,National Highway Performance Program (NHPP),NHPP Exempt,NHPP Exempt,both,False,False
13,Y130,National Electric Vehicle Infrastructure (NEVI) Program,National Electrict Vehicle Formula Program*Amended 2/1/22,National Electrict Vehicle Formula Program,both,False,False
16,Y230,Surface Transportation Block Grant (STBG) Program,STBG Program - Urbanized Areas With Population Over 200K,Surface Transportation Block Grant,both,False,False
17,Y233,Surface Transportation Block Grant (STBG) Program,STBG Program Off-System Bridge,Surface Transportation Block Grant,both,False,False
19,Y235,"Special Rule for Areas of 5,000 to Less Than 50,000 Population","Special Rule for Areas of 5,000 to Less Than 50,000 Population [NEW]",Special Rule,both,False,False
20,Y236,Surface Transportation Block Grant (STBG) Program,STBG Program - Areas with Population 50K to 200K [NEW],Surface Transportation Block Grant,both,False,False
21,Y237,Surface Transportation Block Grant (STBG) Program,"STBG Program - Areas with Population 5K to 49,999 [NEW]",Surface Transportation Block Grant,both,False,False
22,Y238,Surface Transportation Block Grant (STBG) Program,STBG Program - Areas with Population Less Than 5K [NEW],Surface Transportation Block Grant,both,False,False
24,Y240,Surface Transportation Block Grant (STBG) Program,Surface Transportation Block Grant (STBG) Program Flex,Surface Transportation Block Grant,both,False,False
27,Y300,Transportation Alternatives (Section 133(h)),Transportation Alternatives (Section 133(h)) Flex,Transportation Alternatives,both,False,False


In [15]:
code_merge>>filter(_.new_description.isnull(), _.program_name.notnull())

,iija_program_code,new_description,description,program_name,_merge,compare_descriptions_to_programname,compare_descriptions_cols
133,ER01,NaN,Emergency Suppliment Funding,Emergency Supplement Funding,right_only,False,False
134,Y520,NaN,"Federal Lands Planning Program, Access Planning",Federal Lands,right_only,False,False
135,Y928,NaN,Community Project Funding Congressionally Directed Spending,Community Project Funding Congressionally Directed Spending,right_only,False,False


In [16]:
code_merge._merge.value_counts()

both          133
right_only      3
left_only       0
Name: _merge, dtype: int64

In [17]:
code_merge>>filter(_._merge!='both')

,iija_program_code,new_description,description,program_name,_merge,compare_descriptions_to_programname,compare_descriptions_cols
133,ER01,NaN,Emergency Suppliment Funding,Emergency Supplement Funding,right_only,False,False
134,Y520,NaN,"Federal Lands Planning Program, Access Planning",Federal Lands,right_only,False,False
135,Y928,NaN,Community Project Funding Congressionally Directed Spending,Community Project Funding Congressionally Directed Spending,right_only,False,False


In [18]:
code_merge.sample()

,iija_program_code,new_description,description,program_name,_merge,compare_descriptions_to_programname,compare_descriptions_cols
125,YS60,High Risk Rural Roads Special Rule,High Risk Rural Roads Special Rule,High Risk Rural Roads Special Rule,both,True,True


In [19]:
code_merge.new_description.fillna(code_merge['description'], inplace=True)

In [20]:
code_merge = code_merge.drop(columns={'description' , '_merge', 'compare_descriptions_cols', 'compare_descriptions_to_programname'
                                     })

In [21]:
code_merge

,iija_program_code,new_description,program_name
0,Y001,National Highway Performance Program (NHPP),National Highway Performance Program (NHPP)
1,Y002,National Highway Performance Program (NHPP),NHPP Exempt
2,Y003,Projects to Reduce PM 2.5 Emissions,Projects to Reduce PM 2.5 Emissions
3,Y008,Protective Features,Protective Features
4,Y030,Redistribution of Certain Authorized Funds,Redistribution of Certain Authorized Funds
...,...,...,...
131,Y909,Bridge Replacement and Rehabilitation Program,Bridge Replacement and Rehabilitation Program
132,Y926,Community Project Funding Congressionally Directed Spending,Community Project Funding Congressionally Directed Spending
133,ER01,Emergency Suppliment Funding,Emergency Supplement Funding
134,Y520,"Federal Lands Planning Program, Access Planning",Federal Lands


In [22]:
code_merge>>filter(_.program_name.str.contains('Program'))

,iija_program_code,new_description,program_name
0,Y001,National Highway Performance Program (NHPP),National Highway Performance Program (NHPP)
10,Y110,Bridge Formula Program,Bridge Formula Program
11,Y120,Bridge Formula Program,Bridge Formula Program
12,Y120,Bridge Formula Program,Bridge Formula Program
13,Y130,National Electric Vehicle Infrastructure (NEVI) Program,National Electrict Vehicle Formula Program
49,Y444,Highway Research and Development Program,Highway Research and Development Program
50,Y446,Highway Research and Development Program,Highway Research and Development Program
51,Y448,Highway Research and Deveopment Program,Highway Research and Development Program
53,Y450,Metropolitan Planning Program,Metropolitan Planning Program
54,Y460,National Highway Freight Program (NHFP),National Highway Freight Program (NHFP)


### Add new program codes to the data

In [23]:
df>>group_by(_.project_number)>>summarize(n = _.program_code.nunique())>>filter(_.n>1)>>arrange(-_.n)

,project_number,n
506,5004049,4
1115,6054094,4
38,0010203,3
41,0010206,3
44,0010209,3
...,...,...
1427,X023043,2
1444,X059084,2
1454,X075048,2
1460,X093010,2


In [24]:
## number of projects that have more than one program funding code
len(df>>group_by(_.project_number)>>summarize(n = _.program_code_description.nunique())>>filter(_.n>1)>>arrange(-_.n))

108

In [25]:
## number of projects that have more than 2 program funding codes
len(df>>group_by(_.project_number)>>summarize(n = _.program_code_description.nunique())>>filter(_.n>2)>>arrange(-_.n))

9

In [26]:
## take a look at those 9 projects

morethantwopc = (df>>group_by(_.project_number, _.implementing_agency, _.project_title)>>summarize(n = _.program_code_description.nunique())>>filter(_.n>2)>>arrange(-_.n))

In [27]:
project_numbers_to_check = morethantwopc.project_number.to_list()

In [28]:
project_numbers_to_check

['6054094',
 '0010203',
 '0010206',
 '0010209',
 '00PE023',
 '5002203',
 '5004049',
 '5942283',
 '6066137']

In [29]:
## work on agg script

In [30]:
full = _script_utils.get_clean_data(df, full_or_agg = 'full')

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [31]:
full['project_title'] = full.project_title.str.title()
full['project_description'] = full.project_description.str.title()

In [32]:
full.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,county_name_abbrev,county_name_title,program_code_description_for_description,new_project_title
181,2022-05-05,Y230,Surface Transportation Block Grant,5126019,0422000116L,"Appian Way And Marlesta Road Intersection Safety Improvements: Installation Of A Traffic Signal And Traffic Signage, Including Advanced Warning Signs",13,05,Active,"Appian Way And Marlesta Road Intersection Safety Improvements: Installation Of A Traffic Signal And Traffic Signage, Including Advanced Warning Signs On Appian Way, Pavement Striping And Markings, And High_X000D_\n Visibility Crosswalks.",6,4R - Restoration & Rehabilitation,546678.0,350000.0,L5126MTC,5126.0,Pinole,04,Contra Costa County,Metropolitan Transportation Commission,Metropolitan Transportation Commission,CC,Contra Costa County,Surface Transportation Block Grant,Install Traffic Signals in Pinole


In [33]:
## testing the project code descriptions grouped
test_pc = ((
    full.assign(count=1)
    .groupby('project_number')
    .agg({'program_code_description':lambda x:', and the '
          .join(x.unique())})
    .reset_index()))

In [34]:
test_pc[test_pc['project_number'].isin(project_numbers_to_check)]

,project_number,program_code_description
38,0010203,"Safe and Accessible Transportation Options - Metropolitan Planning, and the Metropolitan Planning Program, and the State Planning and Research"
41,0010206,"Safe and Accessible Transportation Options - Metropolitan Planning, and the Metropolitan Planning Program, and the State Planning and Research"
44,0010209,"Safe and Accessible Transportation Options - Metropolitan Planning, and the Metropolitan Planning Program, and the State Planning and Research"
93,00PE023,"National Highway Performance Program (NHPP), and the Surface Transportation Block Grant, and the Highway Safety Improvement Program (HSIP)"
495,5002203,"Surface Transportation Block Grant, and the Transportation Alternatives, and the Congestion Mitigation & Air Quality Improvement"
506,5004049,"National Highway Performance Program (NHPP), and the Bridge Formula Program, and the Bridge Replacement and Rehabilitation Program"
1034,5942283,"Projects to Reduce PM 2.5 Emissions, and the Surface Transportation Block Grant, and the Congestion Mitigation & Air Quality Improvement"
1115,6054094,"Surface Transportation Block Grant, and the Congestion Mitigation & Air Quality Improvement, and the Carbon Reduction Program, and the Community Project Funding Congressionally Directed Spending"
1121,6066137,"Surface Transportation Block Grant, and the Transportation Alternatives, and the Carbon Reduction Program"


## Test agg

In [35]:
agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

/home/jovyan/data-analyses/dla/iija/_script_utils.py:465: FutureWarning: The default value of regex will change from True to False in a future version.
  df['implementing_agency_locode'] = df['implementing_agency_locode'].str.replace('.0', '')


In [36]:
agg.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
847,2023-03-15,5208182,Clovis,L5208FCOG,Y230,Surface Transportation Block Grant,0623000005L,4,4R - No Added Capacity,"BULLARD AVE FROM ARMSTRONG AVE TO TEMPERANCE AVE ROAD REHABILITATION INCLUDING GRINDING, PAVING, ASPHALT CONCRETE, INSTALLING TRAFFIC DEVICES, AND R",679060,|22|,|06|,19,Fresno County,|FRE|,58,Council of Fresno County Governments,Council Of Fresno County Goverments,Install Road Rehabiliation in Clovis,"Install Road Rehabiliation in Clovis, part of the program(s) Surface Transportation Block Grant. (Federal Project ID: 5208182)."
452,2022-09-12,P025033,California,S AMBAG,YS30,Highway Safety Improvement Program (HSIP),0517000185S,17|21|42,Construction Engineering|Safety|Training,SAN BENITO COUNTY NEAR HOLLISTER ON ROUTE 25 FROM 0.4 MILE EAST TO 0.3 MILE WEST OF ROUTE 156 AND ON ROUTE 156 FROM 0.4 MILE SOUTH TO 0.3 MILE NORTH,9705753,|20|,|05|,69,San Benito County,|SBT|,nan,None,None,Safety Improvements in San Benito County,"Safety Improvements in San Benito County, part of the program(s) Highway Safety Improvement Program (HSIP). (Federal Project ID: P025033)."
327,2022-07-28,5930074,Calaveras County,L5930NON-MPO,Y233,Surface Transportation Block Grant,1012000213L,16,Right of Way,DOGTOWN ROAD OVER INDIAN CREEK (BRIDGE 30C0051) BRIDGE REPLACEMENT (TC),250000,|04|,|10|,9,Calaveras County,|CAL|,59,Calaveras Council of Governments,NON-MPO,Replace Bridge in Calaveras County,"Replace Bridge in Calaveras County, part of the program(s) Surface Transportation Block Grant. (Federal Project ID: 5930074)."
699,2023-01-19,5907073,Lassen County,L5907NON-MPO,YS30,Highway Safety Improvement Program (HSIP),0219000114L,17|21,Construction Engineering|Safety,ON JANESVILLE GRADE COUNTY ROAD 208 AT EIGHT LOCATIONS. REPLACE GUARDRAIL AS SAFETY FUNDS SET-ASIDE PROJECT ALLOWED BY HSIP GUIDELINES FOR CYCLE 9 PR,456300,|01|,|02|,35,Lassen County,|LAS|,57,Lassen County Transportation Commission,NON-MPO,Replace Guardrails in Lassen County,"Replace Guardrails in Lassen County, part of the program(s) Highway Safety Improvement Program (HSIP). (Federal Project ID: 5907073)."
1435,2023-06-27,Q101415,California,S MTC,Y001|Y909,National Highway Performance Program (NHPP)|Bridge Replacement and Rehabilitation Program,0415000004S,6|17|42,4R - Restoration & Rehabilitation|Construction Engineering|Training,SAN MATEO COUNTY IN REDWOOD CITY AND SAN CARLOS FROM 0.3 MILE SOUTH OF CORDILLERAS CREEK BRIDGE TO 0.1 MILES SOUTH OF OF PULGAS CREEK BRIDGE REPLACE,26483900,|14|,|04|,81,San Mateo County,|SM|,nan,None,None,Road Restoration & Rehabilitation in San Mateo County,"Road Restoration & Rehabilitation in San Mateo County, part of the program(s) National Highway Performance Program (NHPP), and the Bridge Replacement and Rehabilitation Program. (Federal Project ID: Q101415)."


In [37]:
agg[agg['project_number'].isin(project_numbers_to_check)]

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,old_project_title_desc,obligations_amount,congressional_district,district,county_code,county_name,county_name_abbrev,implementing_agency_locode,rtpa_name,mpo_name,new_project_title,new_description_col
956,2023-04-19,5002203,Sacramento,L5002SACOG,Y230|Y301|Y400,Surface Transportation Block Grant|Transportation Alternatives|Congestion Mitigation & Air Quality Improvement,0318000098L,4|17,4R - No Added Capacity|Construction Engineering,ON BROADWAY FROM 3RD STREET TO 16TH STREET (BROADWAY COMPLETE STREETS PHASE 1 PROJECT) & FROM 16TH STREET TO 24TH STREET (BROADWAY COMPLETE STEETS PH,10903000,|06|,|03|,67,Sacramento County,|SAC|,02,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,Complete Streets in Sacramento,"Complete Streets in Sacramento, part of the program(s) Surface Transportation Block Grant, and the Transportation Alternatives, and the Congestion Mitigation & Air Quality Improvement. (Federal Project ID: 5002203)."
1187,2023-06-01,6054094,Riverside County Transportation Commission,L6054SCAG,Y230|Y400|Y601|Y926,Surface Transportation Block Grant|Congestion Mitigation & Air Quality Improvement|Carbon Reduction Program|Community Project Funding Congressionally Directed Spending,0819000062L,15,Preliminary Engineering,I-15 FROM CAJALCO ROAD IN THE CITY OF CORONA TO STATE ROUTE 74 IN THE CITY OF LAKE ELSINORE EXTEND TOLL EXPRESS LANES SOUTH IN EACH TRAVEL DIRECTION,40151820,|42|,|08|,65,Riverside County,|RIV|,54,Riverside County Transportation Commission,Southern California Association Of Governments,Extend Express Lanes in Riverside County Transportation Commission,"Extend Express Lanes in Riverside County Transportation Commission, part of the program(s) Surface Transportation Block Grant, and the Congestion Mitigation & Air Quality Improvement, and the Carbon Reduction Program, and the Community Project Funding Congressionally Directed Spending. (Federal Project ID: 6054094)."
1188,2023-06-01,6066137,San Diego Association of Governments,L6066SANDAG,Y230|Y301|Y601,Surface Transportation Block Grant|Transportation Alternatives|Carbon Reduction Program,1118000175L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,ALONG IMPERIAL AVENUE FROM 21ST STREET TO 47TH STREET IN THE CITY OF SAN DIEGO. THE IMPERIAL AVENUE BIKEWAY IS A REGIONAL PRIORITY BIKEWAY CONNECTING,8906000,|51|,|11|,73,San Diego County,|SD|,66,NON-RTPA,San Diego Association Of Governments,Bike Lanes in San Diego Association of Governments,"Bike Lanes in San Diego Association of Governments, part of the program(s) Surface Transportation Block Grant, and the Transportation Alternatives, and the Carbon Reduction Program. (Federal Project ID: 6066137)."
1230,2023-06-14,00PE023,California,S NON-MPO,Y001|Y240|YS30,National Highway Performance Program (NHPP)|Surface Transportation Block Grant|Highway Safety Improvement Program (HSIP),00000PE023S,15,Preliminary Engineering,STATEWIDE STATEWIDE PRELIMINARY ENGINEERING 2022-23,376813864,|01|,|nan|,999,Statewide,|NA|,nan,None,None,Preliminary Engineering Projects Statewide,"Preliminary Engineering Projects Statewide, part of the program(s) National Highway Performance Program (NHPP), and the Surface Transportation Block Grant, and the Highway Safety Improvement Program (HSIP). (Federal Project ID: 00PE023)."
1274,2023-06-16,5942283,Fresno County,L5942FCOG,Y003|Y230|Y400,Projects to Reduce PM 2.5 Emissions|Surface Transportation Block Grant|Congestion Mitigation & Air Quality Improvement,0617000208L,4|17|16,4R - No Added Capacity|Construction Engineering|Right of Way,JENSEN AVENUE FROM DICKERSEN TO MADERA AVENUES SHOULDER IMPROVEMENTS,3937177,|21|,|06|,19,Fresno County,|FRE|,5942,Council of Fresno County Governments,Council Of Fresno County Goverments,Improve Shoulders in

In [38]:
assert len(agg) == df.project_number.nunique()